In [1]:
import pandas as pd
import numpy as np

In [81]:
train = pd.read_csv("median_scaled.csv")

In [82]:
test = pd.read_csv("scaled_test.csv")

In [83]:
train.shape

(6818, 41)

In [84]:
test.shape

(2937, 40)

In [85]:
tra = train.copy()

In [86]:
tes = test.copy()

Drop Attr3, 10, 19, 23, 31 - because of high correlation

In [111]:
train.drop(["Attr3", "Attr10", "Attr19", "Attr23", "Attr31"], axis = 1, inplace = True)

In [112]:
test.drop(["Attr3", "Attr10", "Attr19", "Attr23", "Attr31"], axis = 1, inplace = True)

In [113]:
train.shape

(6818, 36)

In [114]:
test.shape

(2937, 35)

In [115]:
train.to_csv("final_train_data.csv")
test.to_csv("final_test_data.csv")

## Light GBM

In [11]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [151]:
X = train.drop(["class"], axis = 1)
y = train['class'].astype('category')
X['Attr70'] = X['Attr70'].astype('category')

In [87]:
X2 = tra.drop(["class"], axis = 1)
y2 = tra['class'].astype('category')
X2['Attr70'] = X2['Attr70'].astype('category')

In [13]:
lgbm = LGBMClassifier(n_estimators=1000, num_leaves=50, subsample=0.8,
                      min_child_samples=20, max_depth=15)

In [14]:
lgbm.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=50, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.8, subsample_for_bin=200000, subsample_freq=0)

In [152]:
test['Attr70'] = test['Attr70'].astype('category')

In [16]:
pred_y = lgbm.predict(test)

In [17]:
print(pred_y)

[0 0 0 ... 0 0 0]


In [18]:
np.sum(pred_y)

21

In [88]:
lgbm.fit(X2, y2)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=50, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.8, subsample_for_bin=200000, subsample_freq=0)

In [90]:
tes['Attr70'] = tes['Attr70'].astype('category')

In [92]:
lgbm_pred_y_all = lgbm.predict(tes)

In [93]:
np.sum(lgbm_pred_y_all)

18

## XGBoost

In [116]:
from xgboost import XGBClassifier

In [176]:
xgb_model = XGBClassifier(learning_rate = 0.2, n_estimators  = 500)

In [177]:
xgb_model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [178]:
xgb_pred_y = xgb_model.predict(test)

In [179]:
np.sum(xgb_model.predict_proba(test)[:,1]>0.15)

153

In [183]:
xgb_sr = pd.Series(xgb_model.predict_proba(test)[:,1]>0.15)
xgb_sr = xgb_sr.astype("int")

In [184]:
xgb_lst = list(xgb_sr[xgb_sr == True].index)

In [185]:
xgb_sr.to_csv("final_test_y.csv")

In [186]:
np.sum(xgb_sr)

153

In [130]:
np.sum(xgb_pred_y)

671

In [94]:
X2['Attr70'] = X2['Attr70'].astype('int')
tes['Attr70'] = tes['Attr70'].astype('int')

In [95]:
xgb_model.fit(X2,y2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [96]:
xgb_pred_y_all = xgb_model.predict(tes)

In [97]:
np.sum(xgb_pred_y_all)

47

In [25]:
lgbm_sr = pd.Series(pred_y)

In [26]:
xgb_sr = pd.Series(xgb_pred_y)

In [27]:
lgbm_lst = list(lgbm_sr.index[lgbm_sr==1])

In [28]:
xgbm_lst = list(xgb_sr.index[xgb_sr==1])

In [29]:
common = [x for x in xgbm_lst if x in lgbm_lst]

In [30]:
len(lgbm_lst)

21

In [31]:
len(common)

18

## Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
forest = RandomForestClassifier(n_estimators=500, random_state=24)

In [34]:
forest.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=24, verbose=0,
                       warm_start=False)

In [35]:
for_pred_y = forest.predict(test)

In [36]:
np.sum(for_pred_y)

21